<a href="https://colab.research.google.com/github/FutureOfTed/AI-class/blob/main/Week7/CIFAR_10_classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os

In [7]:
# ========================================
# 경로 설정
# ========================================
DATA_DIR = '/content/drive/MyDrive/data/cifar10_data'  # CIFAR-10 데이터를 저장할 경로

In [6]:
# 데이터 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)

In [9]:
# --- CIFAR-10 로드 (자동으로 캐싱) ---
# Keras는 기본적으로 ~/.keras/datasets에 저장하지만,
# 명시적으로 경로를 지정하려면 아래와 같이 처리
cifar10_path = os.path.join(DATA_DIR,'cifar-10-batches-py')

if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터 발견: {cifar10_path}")
    print("저장된 데이터를 로드합니다...")
else:
    print(f"데이터가 없습니다. {DATA_DIR}에 다운로드합니다...")

데이터가 없습니다. /content/drive/MyDrive/data/cifar10_data에 다운로드합니다...


In [10]:
# 데이터 로드 (없으면 자동 다운로드 후 캐싱)
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(f"✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {x_train.shape}")
print(f"  - 테스트 데이터: {x_test.shape}\n")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3)
  - 테스트 데이터: (10000, 32, 32, 3)



In [11]:
# 클래스 이름 정의
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [12]:
# 사용할 클래스 선택
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c)for c in selected_classes]

In [13]:
# --- 해당 클래스만 필터링 ---
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask = np.isin(y_test, selected_idx).flatten()

In [14]:
x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]

In [15]:
print(f"선택된 클래스: {selected_classes}")
print(f"  - 학습 샘플 수: {len(x_train)}")
print(f"  - 테스트 샘플 수: {len(x_test)}\n")

선택된 클래스: ['cat', 'dog', 'horse']
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



In [17]:
# 라벨을 0~2로 다시 매핑
label_map = {v: i for i, v in enumerate(selected_idx)}
y_train = np.array([label_map[int(y)]for y in y_train])
y_test = np.array([label_map[int(y)]for y in y_test])


/tmp/ipython-input-2996602248.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_train = np.array([label_map[int(y)]for y in y_train])
/tmp/ipython-input-2996602248.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_test = np.array([label_map[int(y)]for y in y_test])


In [18]:
# 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

In [19]:
# ========================================
# 2. CNN 모델 생성 (새로 추가된 부분) ⭐️
# ========================================
model_cnn = models.Sequential([
    # 첫 번째 Convolutional block
    # 입력 이미지 크기는 (32, 32, 3)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),

    # 두 번째 Convolutional block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # 세 번째 Convolutional block
    layers.Conv2D(64, (3, 3), activation='relu'),

    # ⭐️ Classification을 위한 DNN(Dense Layer) 부분
    layers.Flatten(), # 3D feature map을 1D 벡터로 펼치기
    layers.Dense(64, activation='relu'),

    # 출력층: 3개 클래스('cat', 'dog', 'horse')를 분류하므로 뉴런 수는 3
    layers.Dense(3, activation='softmax')
])

print("🚀 CNN 모델 구조:")
model_cnn.summary() # 모델 구조 확인

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🚀 CNN 모델 구조:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,115 (477.01 KB)

 Trainable params: 122,115 (477.01 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# ========================================
# 3. 모델 컴파일 및 학습 (CNN 모델 적용)
# ========================================
model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', # 라벨이 정수 형태이므로 사용
                  metrics=['accuracy'])

print("\n CNN 모델 학습을 시작합니다...")
history = model_cnn.fit(x_train, y_train,
                        epochs=20, # 학습 횟수
                        batch_size=64,
                        validation_split=0.2) # 훈련 데이터 중 20%를 검증용으로 사용



 CNN 모델 학습을 시작합니다...
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step - accuracy: 0.6340 - loss: 0.7846 - val_accuracy: 0.6830 - val_loss: 0.7173
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 21s 98ms/step - accuracy: 0.6941 - loss: 0.6775 - val_accuracy: 0.6940 - val_loss: 0.6705
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 19s 90ms/step - accuracy: 0.7210 - loss: 0.6420 - val_accuracy: 0.7023 - val_loss: 0.6758
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 17s 90ms/step - accuracy: 0.7425 - loss: 0.5960 - val_accuracy: 0.7013 - val_loss: 0.6969
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 22s 96ms/step - accuracy: 0.7602 - loss: 0.5531 - val_accuracy: 0.7213 - val_loss: 0.6437
Epoch 6/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 20s 91ms/step - accuracy: 0.7674 - loss: 0.5346 - val_accuracy: 0.7377 - val_loss: 0.6201
Epoch 7/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 17s 90ms/step - accuracy: 0.7964 - loss: 0.4861 - val_accuracy: 0.7407 - val_loss: 0.6416
Epoch 8/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 17s 91ms/step - accuracy: 0.

In [22]:
# ========================================
# 4. 모델 평가 (CNN 모델 평가)
# ========================================
print("\n 모델 평가 중...")
test_loss, test_acc = model_cnn.evaluate(x_test, y_test, verbose=2)
print(f"\n최종 테스트 정확도: {test_acc:.4f}")


 모델 평가 중...
94/94 - 1s - 12ms/step - accuracy: 0.7223 - loss: 1.2392

최종 테스트 정확도: 0.7223
